----------Librerias utilizadas----------    

In [1]:
pip install qvd

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [7]:
import re
import pandas as pd
from qvd import qvd_reader

----------lectura y filtro del excel. Pasa al dfDatos----------

In [196]:
ruta = "./Parametros1.xlsx"
# Variable de la ruta relativa.
dfxls = pd.read_excel(ruta, sheet_name="Validaciones")
# Asignación de la hoja de excel Validaciones a un df
dfDatos = dfxls[dfxls["Tipo Validacion"].isin(["Formato"])]
# Se filtra por el tipo de validación Formato
dfDatos.head()

,Esquema,Tabla,Campo,Tipo Validacion,Valores,Prioridad,Dimension,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
3,dbo,SalesOrderDetail,CarrierTrackingNumber,Formato,^\w{4}-\w{4}-\w{2}$,NaN,NaN,NaN,NaN,NaN,NaN
4,dbo,SalesOrderDetail,LlaveCompuestaD,Formato,"^[-@.\/|\w\s]{16,31}$",NaN,NaN,NaN,NaN,NaN,NaN


----------Creacion de rutas que se usan en el codigo----------

In [197]:
dfRuta = pd.read_excel(ruta, sheet_name="Ejecucion")
#Lee la hoja Ejexcución del EXCEL
dfRutaCSV = dfRuta[dfRuta["Parametro"].isin(["PathPython"])]
#Busca en parametros el que contiene el PathPython
dfRutaCSV1 = dfRuta[dfRuta["Parametro"].isin(["PathDestin"])]
dfRutaCSV = dfRutaCSV.reset_index()
#Resetea los indices del dfRustas para facilitar la busqueda
dfRutaCSV1


,Parametro,Valor,Observaciones
6,PathDestin,C:\QVD\Destino,Ruta local indicando ruta para guardar informa...


-------- Ciclo for recorre el df datos en busca de las tablas a validar y lectura de qvd---------

In [198]:
#Ciclo lee el dfDatos en busca de las tablas a validar
#El nombre del qvd se guarda en la variable qvd
tablas = dfDatos.drop_duplicates(subset=['Esquema', 'Tabla'])
name_tablas = tablas.reset_index()
name_tabla = name_tablas.iloc[0]["Tabla"]
df_param_ = dfDatos.loc[dfDatos['Tabla'] == name_tabla]
for i in range(len(df_param_)):
    tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
    pathCSV1 = (dfRutaCSV1.iloc[0]["Valor"]) 
    qvd = pathCSV1+ "\\"  +tabla +".qvd"    
    #print(tabla)
    #print(pathCSV1)
    dfqvd = qvd_reader.read(qvd)
    dfqvd_copy=dfqvd

In [199]:
dfqvd['LlaveCompuestaD'][121316]= 'SalesOrderDetail|121317|75123|1|'

In [200]:
dfqvd

,LlaveCompuestaD,SalesOrderDetailID,SalesOrderID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,SalesOrderDetail|1|43659|1,1,43659,4911-403C-98,1,776,1,2024.994,0,2024.994,{B207C96D-D9E6-402B-8470-2CC176C42283},40694
1,SalesOrderDetail|2|43659|1,2,43659,4911-403C-98,3,777,1,2024.994,0,6074.982,{7ABB600D-1E77-41BE-9FE5-B9142CFC08FA},40694
2,SalesOrderDetail|3|43659|1,3,43659,4911-403C-98,1,778,1,2024.994,0,2024.994,{475CF8C6-49F6-486E-B0AD-AFC6A50CDD2F},40694
3,SalesOrderDetail|4|43659|1,4,43659,4911-403C-98,1,771,1,2039.994,0,2039.994,{04C4DE91-5815-45D6-8670-F462719FBCE3},40694
4,SalesOrderDetail|5|43659|1,5,43659,4911-403C-98,1,772,1,2039.994,0,2039.994,{5A74C7D2-E641-438E-A7AC-37BF23280301},40694
...,...,...,...,...,...,...,...,...,...,...,...,...
121312,SalesOrderDetail|121313|75122|1,121313,75122,None,1,878,1,21.98,0,21.98,{8CAD6675-18CC-4F47-8287-97B41A8EE47D},41820
121313,SalesOrderDetail|121314|75122|1,121314,75122,None,1,712,1,8.99,0,8.99,{84F1C363-1C50-4442-BE16-541C59B6E12C},41820
121314,SalesOrderDetail|121315|75123|1,121315,75123,None,1,878,1,21.98,0,21.98,{C18B6476-429F-4BB1-828E-2BE5F82A0A51},41820
121315,SalesOrderDetail|121316|75123|1,121316,75123,None,1,879,1,159,0,159,{75A89C6A-C60A-47EA-8A52-B52A9C435B64},41820


----------Creacion de variables vacias que se llenaran con las validaciones de expresiones regulares----------

In [201]:
#Variable conteo para guardar el resultado del Groupby
dfqvd["ValoresValidacion"] = ""
#Creacion de variable esquema que va a contener tabla.campo
dfqvd["Campo"] = ""

----------Validación de expresión regular----------

In [202]:
# Ciclo recorre el dfDatos y asigna a las variables campo, Formato y tabla el contenido de dfDatos para la validación
listas1 =[]

for i in range(len(df_param_)):
    campo = (df_param_.iloc[i]["Campo"])
    Formato = (df_param_.iloc[i]["Valores"])
    tabla = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"])
    print(tabla, campo)
    print(tabla+"."+campo)
    dfqvd[tabla+"."+campo] = dfqvd[campo]
    dfqvd = dfqvd.replace({campo: Formato},{campo:"Valido"}, regex=True)
    tabla_campo = (df_param_.iloc[i]["Esquema"]+"_"+df_param_.iloc[i]["Tabla"]+"."+df_param_.iloc[i]["Campo"])    
    #Lleamos los valores nulos con "No valido"
    dfqvd1 = dfqvd.fillna("Nulo")
    listas1.append([tabla_campo,campo])
    #print(campo)
    #print(Formato)
    #print(tabla)
    #print(listas1)
dfqvd1


dbo_SalesOrderDetail CarrierTrackingNumber
dbo_SalesOrderDetail.CarrierTrackingNumber
dbo_SalesOrderDetail LlaveCompuestaD
dbo_SalesOrderDetail.LlaveCompuestaD


,LlaveCompuestaD,SalesOrderDetailID,SalesOrderID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate,ValoresValidacion,Campo,dbo_SalesOrderDetail.CarrierTrackingNumber,dbo_SalesOrderDetail.LlaveCompuestaD
0,Valido,1,43659,Valido,1,776,1,2024.994,0,2024.994,{B207C96D-D9E6-402B-8470-2CC176C42283},40694,,,4911-403C-98,SalesOrderDetail|1|43659|1
1,Valido,2,43659,Valido,3,777,1,2024.994,0,6074.982,{7ABB600D-1E77-41BE-9FE5-B9142CFC08FA},40694,,,4911-403C-98,SalesOrderDetail|2|43659|1
2,Valido,3,43659,Valido,1,778,1,2024.994,0,2024.994,{475CF8C6-49F6-486E-B0AD-AFC6A50CDD2F},40694,,,4911-403C-98,SalesOrderDetail|3|43659|1
3,Valido,4,43659,Valido,1,771,1,2039.994,0,2039.994,{04C4DE91-5815-45D6-8670-F462719FBCE3},40694,,,4911-403C-98,SalesOrderDetail|4|43659|1
4,Valido,5,43659,Valido,1,772,1,2039.994,0,2039.994,{5A74C7D2-E641-438E-A7AC-37BF23280301},40694,,,4911-403C-98,SalesOrderDetail|5|43659|1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121312,Valido,121313,75122,Nulo,1,878,1,21.98,0,21.98,{8CAD6675-18CC-4F47-8287-97B41A8EE47D},41820,,,Nulo,SalesOrderDetail|121313|75122|1
121313,Valido,121314,75122,Nulo,1,712,1,8.99,0,8.99,{84F1C363-1C50-4442-BE16-541C59B6E12C},41820,,,Nulo,SalesOrderDetail|121314|75122|1
121314,Valido,121315,75123,Nulo,1,878,1,21.98,0,21.98,{C18B6476-429F-4BB1-828E-2BE5F82A0A51},41820,,,Nulo,SalesOrderDetail|121315|75123|1
121315,Valido,121316,75123,Nulo,1,879,1,159,0,159,{75A89C6A-C60A-47EA-8A52-B52A9C435B64},41820,,,Nulo,SalesOrderDetail|121316|75123|1


----------Captura de tabla_campo y campo ----------

In [203]:
listas1

[['dbo_SalesOrderDetail.CarrierTrackingNumber', 'CarrierTrackingNumber'],
 ['dbo_SalesOrderDetail.LlaveCompuestaD', 'LlaveCompuestaD']]

In [204]:
campo_var = pd.DataFrame(listas1,columns=["ValoresCampo","Cruza"])
campo_var

,ValoresCampo,Cruza
0,dbo_SalesOrderDetail.CarrierTrackingNumber,CarrierTrackingNumber
1,dbo_SalesOrderDetail.LlaveCompuestaD,LlaveCompuestaD


----------Creacion un DataFrame que servirá como base para el .CSV final ----------

In [205]:
dfGroup=pd.DataFrame({"Campo":[],"ValoresCampo":[],"Cruza":[],"ValoresValidacion":[]})
dfGroup

,Campo,ValoresCampo,Cruza,ValoresValidacion


----------Funcion creada para dar como no validas los registros que no cumplen con la extresion regular----------

In [207]:
def asd (a):
    if a != "Valido":
        return("No valido")
    else:
        return(a)

----------Ciclo for en el que se agrupan por Campo, ValoresCampo Cruza y ValoresValidacion y se concatenan los resultados de cada formato requerido----------

In [195]:
for i in range(len(df_param_)):
    dfGroup11 = dfqvd1.groupby(["Campo",campo_var.iloc[i]["ValoresCampo"] ,campo_var.iloc[i]["Cruza"] ],as_index= False).agg({"ValoresValidacion":"count"})
    dfGroup11.rename(columns={"Campo":"Campo",campo_var.iloc[i]["ValoresCampo"]:"ValoresCampo",campo_var.iloc[i]["Cruza"]:"Cruza","ValoresValidacion":"ValoresValidacion"},inplace= True )
    dfGroup11["Campo"] = campo_var.iloc[i]["ValoresCampo"]   
    dfGroup = pd.concat([dfGroup,dfGroup11],axis=0)
    dfGroup["ValoresCampo"] = dfGroup["ValoresCampo"].replace("Nulo","")
    dfGroup["Cruza"] = dfGroup["Cruza"].apply(lambda a: asd(a))
    #dfGroup["ValoresValidacion"] = int(dfGroup["ValoresValidacion"])
dfGroup

,Campo,ValoresCampo,Cruza,ValoresValidacion
0,dbo_SalesOrderDetail.CarrierTrackingNumber,000A-434D-BC,Valido,20.0
1,dbo_SalesOrderDetail.CarrierTrackingNumber,0031-4C10-84,Valido,39.0
2,dbo_SalesOrderDetail.CarrierTrackingNumber,0083-444C-8D,Valido,1.0
3,dbo_SalesOrderDetail.CarrierTrackingNumber,0097-43FD-B7,Valido,8.0
4,dbo_SalesOrderDetail.CarrierTrackingNumber,00A2-4AD1-88,Valido,1.0
...,...,...,...,...
121312,dbo_SalesOrderDetail.LlaveCompuestaD,SalesOrderDetail|99999|68516|1,Valido,1.0
121313,dbo_SalesOrderDetail.LlaveCompuestaD,SalesOrderDetail|9999|46355|1,Valido,1.0
121314,dbo_SalesOrderDetail.LlaveCompuestaD,SalesOrderDetail|999|43894|1,Valido,1.0
121315,dbo_SalesOrderDetail.LlaveCompuestaD,SalesOrderDetail|99|43668|1,Valido,1.0


----------Exportar a csv----------

In [143]:
pathCSV = (dfRutaCSV.iloc[0]["Valor"]) 
pathCSV = pathCSV + "\\"  +tabla + ".csv"
#print(pathCSV)

In [144]:
dfGroup.to_csv(pathCSV, index=False)

----------Mensaje final----------

In [145]:
print("Archivo exportado, revisar ruta  " + pathCSV)

Archivo exportado, revisar ruta  C:\QVD\Python\dbo_SalesOrderDetail.csv


In [146]:
dfqvd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121317 entries, 0 to 121316
Data columns (total 16 columns):
 #   Column                                      Non-Null Count   Dtype 
---  ------                                      --------------   ----- 
 0   LlaveCompuestaD                             121317 non-null  object
 1   SalesOrderDetailID                          121317 non-null  object
 2   SalesOrderID                                121317 non-null  object
 3   CarrierTrackingNumber                       60919 non-null   object
 4   OrderQty                                    121317 non-null  object
 5   ProductID                                   121317 non-null  object
 6   SpecialOfferID                              121317 non-null  object
 7   UnitPrice                                   121317 non-null  object
 8   UnitPriceDiscount                           121317 non-null  object
 9   LineTotal                                   121317 non-null  object
 10  rowguid 

In [183]:
dfqvd["dbo_Person_Person.LlaveCompuestaD"] = dfqvd["LlaveCompuestaD"]
dfqvd = dfqvd.replace({"LlaveCompuestaD": "MaxNumRegistros"},{"LlaveCompuestaD":"Valido"}, regex=True)
dfqvd

,LlaveCompuestaD,SalesOrderDetailID,SalesOrderID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate,121316,ValoresValidacion,Campo,dbo_SalesOrderDetail.CarrierTrackingNumber,dbo_SalesOrderDetail.LlaveCompuestaD,dbo_Person_Person.LlaveCompuestaD
0,Valido,1,43659,Valido,1,776,1,2024.994,0,2024.994,{B207C96D-D9E6-402B-8470-2CC176C42283},40694,SalesOrderDetail|121317|75123|1|,,,4911-403C-98,SalesOrderDetail|1|43659|1,Valido
1,Valido,2,43659,Valido,3,777,1,2024.994,0,6074.982,{7ABB600D-1E77-41BE-9FE5-B9142CFC08FA},40694,SalesOrderDetail|121317|75123|1|,,,4911-403C-98,SalesOrderDetail|2|43659|1,Valido
2,Valido,3,43659,Valido,1,778,1,2024.994,0,2024.994,{475CF8C6-49F6-486E-B0AD-AFC6A50CDD2F},40694,SalesOrderDetail|121317|75123|1|,,,4911-403C-98,SalesOrderDetail|3|43659|1,Valido
3,Valido,4,43659,Valido,1,771,1,2039.994,0,2039.994,{04C4DE91-5815-45D6-8670-F462719FBCE3},40694,SalesOrderDetail|121317|75123|1|,,,4911-403C-98,SalesOrderDetail|4|43659|1,Valido
4,Valido,5,43659,Valido,1,772,1,2039.994,0,2039.994,{5A74C7D2-E641-438E-A7AC-37BF23280301},40694,SalesOrderDetail|121317|75123|1|,,,4911-403C-98,SalesOrderDetail|5|43659|1,Valido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121312,Valido,121313,75122,None,1,878,1,21.98,0,21.98,{8CAD6675-18CC-4F47-8287-97B41A8EE47D},41820,SalesOrderDetail|121317|75123|1|,,,None,SalesOrderDetail|121313|75122|1,Valido
121313,Valido,121314,75122,None,1,712,1,8.99,0,8.99,{84F1C363-1C50-4442-BE16-541C59B6E12C},41820,SalesOrderDetail|121317|75123|1|,,,None,SalesOrderDetail|121314|75122|1,Valido
121314,Valido,121315,75123,None,1,878,1,21.98,0,21.98,{C18B6476-429F-4BB1-828E-2BE5F82A0A51},41820,SalesOrderDetail|121317|75123|1|,,,None,SalesOrderDetail|121315|75123|1,Valido
121315,Valido,121316,75123,None,1,879,1,159,0,159,{75A89C6A-C60A-47EA-8A52-B52A9C435B64},41820,SalesOrderDetail|121317|75123|1|,,,None,SalesOrderDetail|121316|75123|1,Valido


In [166]:
dfqvd_copy.loc[121316]

LlaveCompuestaD                                      SalesOrderDetail|121317|75123|1
SalesOrderDetailID                                                            121317
SalesOrderID                                                                   75123
CarrierTrackingNumber                                                           None
OrderQty                                                                           1
ProductID                                                                        712
SpecialOfferID                                                                     1
UnitPrice                                                                       8.99
UnitPriceDiscount                                                                  0
LineTotal                                                                       8.99
rowguid                                       {73646D26-0461-450D-8019-2C6C858619CA}
ModifiedDate                                                     